## In this Notebook, we will create a basic Q&A chain and Agent over a SQL database

### Architecture

At a high-level, the steps of any SQL chain and agent are:
* Convert question to SQL query using the LLM Model.
* Execute the SQL query.
* Getting the Model to respond to user input using the query results.

In [2]:
import os
from langchain.utilities import SQLDatabase
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import create_sql_query_chain
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [4]:
llm = GoogleGenerativeAI(
  model="gemini-1.5-pro-latest",
  temperature=0.5,
  google_api_key=os.getenv("GOOGLE_API_KEY"), 
)

In [ ]:
host = 'localhost'
port = '3306'
username = 'root'
password = 'xxxxx'
database_schema = 'analytics'
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=2)
chain = create_sql_query_chain(llm, db)

In [ ]:
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT count(*) FROM customer LIMIT 10;")

In [ ]:
response = chain.invoke({"question": "How many employees are there"})
response

In [ ]:
db.run(response)

In [ ]:
response = chain.invoke({"question": "How many customers are from Bronx"})
print(response)
print(db.run(response))

In [ ]:
response = chain.invoke({"question": "Which state are customers most from?"})
print(response)
print(db.run(response))

In [ ]:
response = chain.invoke({"question": "Give me the top 5 customers who paid the highest shipping charges"})
print(response)
print(db.run(response))